In [2]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [3]:
week = 22 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

20 March 2023 ; 23:14:26


In [4]:
features = ['DWS','DBPM','STL','BLK','DRB','PA/G','BLK%','DRtg/100','MP']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [5]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/dpoy/dpoy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,32,0,9.4,1.5,3.4,0.440,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,1,1
1,Bobby Portis,PF,27,MIL,60,19,26.0,5.7,11.3,0.501,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,0,1
2,Brook Lopez,C,34,MIL,69,69,30.7,6.0,11.5,0.521,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,0,0
3,Giannis Antetokounmpo,PF,28,MIL,56,56,32.5,11.1,20.4,0.546,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,0,0
4,Grayson Allen,SG,27,MIL,65,63,27.8,3.5,7.9,0.441,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,0,0


In [6]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [7]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'DBPM', 'STL', 'BLK', 'DRtg/100', 'DWS', 'Seed', 'Share']]
res.head(5)

,Player,Tm,DBPM,STL,BLK,DRtg/100,DWS,Seed,Share
0,Jaren Jackson Jr.,MEM,2.3,1.0,3.1,104,5.105660,2,0.191508
1,Nikola Jokić,DEN,4.4,1.2,0.7,110,4.356250,1,0.170145
2,Giannis Antetokounmpo,MIL,2.6,0.7,0.8,107,4.978571,1,0.111366
3,Anthony Davis,LAL,1.8,1.0,2.1,108,4.991304,9,0.109893
4,Jimmy Butler,MIA,2.0,1.9,0.4,111,3.958621,7,0.104435


In [8]:
res.to_csv(path + f'Results/dpoy/results_week_{week}.csv', index = None)